<a href="https://colab.research.google.com/github/zloelias/final-project/blob/master/character_level_rotten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#! pip install tensorflow_text
#! pip install textattack['tensorflow']
#! pip install datasets --upgrade
#! pip install captum

In [ ]:
# При установке из репозитория при работе не подцепляются другие модели, работает только дефолтный bert-base-uncased

! pip install tensorflow_text
! git clone https://github.com/zloelias/TextAttack.git
! pip install /content/TextAttack[tensorflow] --use-feature=in-tree-build
! pip install datasets --upgrade
! pip install captum

     |████████████████████████████████| 4.9 MB 4.3 MB/s 
Cloning into 'TextAttack'...
remote: Enumerating objects: 20340, done.
remote: Counting objects: 100% (2625/2625), done.
remote: Compressing objects: 100% (977/977), done.
remote: Total 20340 (delta 1782), reused 2334 (delta 1647), pack-reused 17715
Receiving objects: 100% (20340/20340), 25.42 MiB | 10.46 MiB/s, done.
Resolving deltas: 100% (15055/15055), done.
Processing ./TextAttack
     |████████████████████████████████| 60 kB 3.0 MB/s 
     |████████████████████████████████| 322 kB 8.7 MB/s 
     |████████████████████████████████| 769 kB 47.3 MB/s 
     |████████████████████████████████| 298 kB 57.8 MB/s 
     |████████████████████████████████| 3.3 MB 64.7 MB/s 
     |████████████████████████████████| 101 kB 13.5 MB/s 
     |████████████████████████████████| 454.3 MB 13 kB/s 
     |████████████████████████████████| 124 kB 68.0 MB/s 
     |████████████████████████████████| 462 kB 68.4 MB/s 
     |██████████████████████████████

In [ ]:
import torch
from copy import deepcopy

from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer

#from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients, LayerDeepLiftShap, InternalInfluence, LayerGradientXActivation
#from captum.attr import visualization as viz

import nltk
nltk.download('averaged_perceptron_tagger_ru')
nltk.download('stopwords')

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:44<00:00, 10.9MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpcv_zxgs3.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else: 
    device = torch.device("cpu")
    
print(device)

cuda:0


In [ ]:
from textattack import Attack
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder, MultilingualUniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR, GreedySearch
from textattack.transformations import (
    CompositeTransformation,
    #WordSwapEmbedding,
    WordSwapHomoglyphSwap,
    WordSwapQWERTY,
    WordSwapNeighboringCharacterSwap,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
)

from textattack.constraints.pre_transformation.max_word_index_modification import MaxWordIndexModification
from textattack.constraints.pre_transformation.max_modification_rate import MaxModificationRate

from textattack.attack_recipes.attack_recipe import AttackRecipe


In [ ]:
class TextBuggerLi2018RU(AttackRecipe):
    """Li, J., Ji, S., Du, T., Li, B., and Wang, T. (2018).
    TextBugger: Generating Adversarial Text Against Real-world Applications.
    https://arxiv.org/abs/1812.05271
    """

    @staticmethod
    def build(model_wrapper, max_rate, max_seq_len, language):
        #
        #  we propose five bug generation methods for TEXTBUGGER:
        #
        transformation = CompositeTransformation(
            [
                # (1) Insert: Insert a space into the word.
                # Generally, words are segmented by spaces in English. Therefore,
                # we can deceive classifiers by inserting spaces into words.
                WordSwapRandomCharacterInsertion(
                    random_one=True,
                    letters_to_insert=" ",
                    skip_first_char=True,
                    skip_last_char=True,
                ),
                # (2) Delete: Delete a random character of the word except for the first
                # and the last character.
                WordSwapRandomCharacterDeletion(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                # (3) Swap: Swap random two adjacent letters in the word but do not
                # alter the first or last letter. This is a common occurrence when
                # typing quickly and is easy to implement.
                WordSwapNeighboringCharacterSwap(
                    random_one=True, skip_first_char=True, skip_last_char=True
                ),
                # (4) Substitute-C (Sub-C): Replace characters with visually similar
                # characters (e.g., replacing “o” with “0”, “l” with “1”, “a” with “@”)
                # or adjacent characters in the keyboard (e.g., replacing “m” with “n”).
                WordSwapHomoglyphSwap(),
                # (5) Substitute-W
                # (Sub-W): Replace a word with its topk nearest neighbors in a
                # context-aware word vector space. Specifically, we use the pre-trained
                # GloVe model [30] provided by Stanford for word embedding and set
                # topk = 5 in the experiment.
                #WordSwapEmbedding(max_candidates=5),
                WordSwapQWERTY(random_one=True, skip_first_char=True, skip_last_char=True),
            ]
        )

        constraints = [RepeatModification(), StopwordModification(language=language)]
        constraints.append(MaxWordIndexModification(max_seq_len))
        constraints.append(MaxModificationRate(max_rate, min_threshold=1))

        # In our experiment, we first use the Universal Sentence
        # Encoder [7], a model trained on a number of natural language
        # prediction tasks that require modeling the meaning of word
        # sequences, to encode sentences into high dimensional vectors.
        # Then, we use the cosine similarity to measure the semantic
        # similarity between original texts and adversarial texts.
        # ... "Furthermore, the semantic similarity threshold \eps is set
        # as 0.8 to guarantee a good trade-off between quality and
        # strength of the generated adversarial text."
        constraints.append(MultilingualUniversalSentenceEncoder(
            threshold=0.936338023,
            metric="cosine",
            compare_against_original=True,
            window_size=15,
            skip_text_shorter_than_window=True,
        ))
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)




In [ ]:
import textattack
import random
import numpy as np

experiments = {
    'simple_char_level_attack': [
#        {
#            'dataset': {'name': 'kinopoisk', 'url': 'zloelias/kinopoisk-reviews'},
#            'model': ('cointegrated/rubert-tiny2', 'zloelias/rubert-tiny2-kinopoisk-reviews-finetuned-clf'),
#            'language': ('ru', 'rus', 'russian'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 312,
#            'columns': (['text'], 'labels'),
#        },
#        {
#            'dataset': {'name': 'lenta', 'url': 'zloelias/lenta-ru'},
#            'model': ('cointegrated/rubert-tiny2', 'zloelias/rubert-tiny2-lenta-ru-finetuned-clf'),
#            'language': ('ru', 'rus', 'russian'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 312,
#            'columns': (['text'], 'labels'),
#        },
#        {
#            'dataset': {'name': 'ag_news', 'url': 'ag_news'},
#            'model': ('bert-base-uncased', 'textattack/bert-base-uncased-ag-news'),
#            'language': ('en', 'eng', 'english'),
#            'max_rate': np.linspace(0.1, 0.9, 9),
#            'max_seq_len': 512,
#            'columns': (['text'], 'label'),
#        },
        {
            'dataset': {'name': 'rotten_tomatoes', 'url': 'rotten_tomatoes'},
            'model': ('bert-base-uncased', 'textattack/bert-base-uncased-rotten-tomatoes'),
            'language': ('en', 'eng', 'english'),
            'max_rate': np.linspace(0.1, 0.9, 9),
            'max_seq_len': 512,
            'columns': (['text'], 'label'),
        },
    ]
}

for attack_name in experiments.keys():
    for attack_data in experiments[attack_name]:
        dataset_name, dataset_url = [attack_data['dataset'][k] for k in attack_data['dataset'].keys()]
        dataset_columns = attack_data['columns']
        dataset = HuggingFaceDataset(dataset_url, None, 'test', dataset_columns=dataset_columns)

        base_model_name, model_name = attack_data['model']
        original_model = AutoModelForSequenceClassification.from_pretrained(model_name)
        original_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
        model = HuggingFaceModelWrapper(original_model,original_tokenizer)

        la, lan, language = attack_data['language']
        for max_rate in attack_data['max_rate']:
            experiment_result_dir = f'/content/drive/MyDrive/Colab Notebooks/final-project/experiments/{attack_name}/{dataset_name}/{max_rate}/'
            print(experiment_result_dir)
            attack = TextBuggerLi2018RU.build(model, max_rate, attack_data['max_seq_len'], language)

            attack_args = textattack.AttackArgs(
                num_examples=500, 
                log_to_csv=experiment_result_dir + "log.csv", 
                checkpoint_interval=50, 
                checkpoint_dir=experiment_result_dir + "checkpoints", 
                disable_stdout=True
            )
            attacker = textattack.Attacker(attack, dataset, attack_args)
            attacker.attack_dataset()
        #print(attack_data)
        #print(model_name, model_url)

Downloading:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/921 [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/488k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

...

[Succeeded / Failed / Skipped / Total] 223 / 62 / 65 / 350:  70%|███████   | 350/500 [02:38<01:07,  2.21it/s]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/simple_char_level_attack/rotten_tomatoes/0.9/checkpoints/1639409906896.ta.chkpt" at 2021-12-13 15:38:26 after 350 attacks.


[Succeeded / Failed / Skipped / Total] 253 / 75 / 72 / 400:  80%|████████  | 400/500 [03:03<00:45,  2.18it/s]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/simple_char_level_attack/rotten_tomatoes/0.9/checkpoints/1639409931895.ta.chkpt" at 2021-12-13 15:38:51 after 400 attacks.


[Succeeded / Failed / Skipped / Total] 279 / 88 / 83 / 450:  90%|█████████ | 450/500 [03:23<00:22,  2.21it/s]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/simple_char_level_attack/rotten_tomatoes/0.9/checkpoints/1639409951492.ta.chkpt" at 2021-12-13 15:39:11 after 450 attacks.
[Succeeded / Failed / Skipped / Total] 279 / 88 / 84 / 451:  90%|█████████ | 451/500 [03:23<00:22,  2.22it/s]

[Succeeded / Failed / Skipped / Total] 312 / 94 / 94 / 500: 100%|██████████| 500/500 [03:45<00:00,  2.22it/s]textattack: Saving checkpoint under "/content/drive/MyDrive/Colab Notebooks/final-project/experiments/simple_char_level_attack/rotten_tomatoes/0.9/checkpoints/1639409973503.ta.chkpt" at 2021-12-13 15:39:33 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 312 / 94 / 94 / 500: 100%|██████████| 500/500 [03:45<00:00,  2.22it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 312    |
| Number of failed attacks:     | 94     |
| Number of skipped attacks:    | 94     |
| Original accuracy:            | 81.2%  |
| Accuracy under attack:        | 18.8%  |
| Attack success rate:          | 76.85% |
| Average perturbed word %:     | 23.0%  |
| Average num. words per input: | 18.78  |
| Avg num queries:              | 46.75  |
+-------------------------------+--------+
